# IMPORT AND SEED

In [ ]:
# Model
!git clone "https://github.com/cybernetic-m/eai-project.git" # type: ignore

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR

# Others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

sys.path.append('/content/eai-project/training')
sys.path.append('/content/eai-project/preprocessing')
sys.path.append('/content/eai-project/dataset')
sys.path.append('/content/eai-project/utils')
sys.path.append('/content/eai-project/models')
sys.path.append('/content/eai-project/modules')
sys.path.append('/content/eai-project/testing')
sys.path.append('/content/eai-project')
from train import train
from preprocessing import *
from thermal_dataset import thermal_dataset
from csv_utils import *
from complete_model import complete_model
from testing.test import test
from blocks import mlp, linear, rnn, lstm
from calculate_metrics import calculate_metrics
prefix = '/content'

pd.set_option('display.max_columns', None)

# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

Cloning into 'eai-project'...
remote: Enumerating objects: 884, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (135/135), done.


# INITIALIZE THE MODEL CLASS

In [ ]:
# Need to specify a model.pt file to load! In the directory results that you want to test
result_path = '/eai-project/script_simple/auto-weighted'
model_path = prefix + result_path +'/model.pt'

# Loss function
loss_fn = nn.MSELoss()

# Read hyperparameters
with open (prefix + result_path + '/hyperparam.json', 'r') as f:
    hyperparams = json.load(f)

with open (prefix + result_path + '/ensemble.json', 'r') as f:
    model_dict = json.load(f)
with open(prefix + result_path + '/autoencoder.json', 'r') as f:
    autoencoder_dict = json.load(f)
# Read Ensemble model parameters and initialize the model
if hyperparams['extractor_type'] == 'conv':
    model = complete_model(model_dict=model_dict,
                            device=device,
                            autoencoder_dim=autoencoder_dict['in_kern_out'],
                            pooling_kernel_size=autoencoder_dict['pooling_kernel_size'],
                            padding=autoencoder_dict['padding'],
                            pooling=autoencoder_dict['pooling'],
                            scale_factor = autoencoder_dict['scale_factor'],
                            upsample_mode=autoencoder_dict['upsample_mode'],
                            dropout=autoencoder_dict['dropout'],
                            mode=hyperparams['mode'],
                            heterogeneous=hyperparams['heterogeneous'],
                            timesteps=hyperparams['timesteps'],
                            norm=hyperparams['norm']
                            ).to(device)

elif 'lstm' in hyperparams['extractor_type']:
    model = complete_model(
                                model_dict=model_dict,
                                device=device,
                                timesteps=hyperparams['timesteps'],
                                lstm_layers=autoencoder_dict['lstm_layers'],
                                autoencoder_dim=autoencoder_dict['in_hidd'],
                                dropout=autoencoder_dict['dropout'],
                                extractor_type=hyperparams['extractor_type'],
                                heterogeneous=hyperparams['heterogeneous'],
                                norm=hyperparams['norm'],
                                mode=hyperparams['mode']
                                ).to(device)

print(hyperparams)
#print("state_dict",model.state_dict().keys())

autoencoder = hyperparams['extractor_type'] == 'conv' or hyperparams['extractor_type'] == 'lstm_autoencoder'

Autoencoder type: LSTM
Autoencoder Summary: lstm_autoencoder(
  (encoder): lstm_encoder(
    (lstm_layers): ModuleList(
      (0): lstm(
        (lstm): LSTM(4, 2, num_layers=2, batch_first=True, dropout=0.15)
      )
    )
  )
  (decoder): lstm_decoder(
    (lstm_layers): ModuleList(
      (0): lstm(
        (lstm): LSTM(2, 4, num_layers=2, batch_first=True, dropout=0.15)
      )
    )
    (linear_decoder): Linear(in_features=800, out_features=800, bias=True)
  )
)
Ensemble Model Summary: ModuleList(
  (0): mlp(
    (linear_layers): ModuleList(
      (0): Linear(in_features=403, out_features=604, bias=True)
      (1): Linear(in_features=604, out_features=201, bias=True)
      (2): Linear(in_features=201, out_features=100, bias=True)
      (3): Linear(in_features=100, out_features=67, bias=True)
      (4): Linear(in_features=67, out_features=50, bias=True)
      (5): Linear(in_features=50, out_features=40, bias=True)
      (6): Linear(in_features=40, out_features=33, bias=True)
      (

# DOWNLOAD AND UNZIP

In [ ]:
link_zipped_csv = 'https://drive.google.com/file/d/1MssQF4pI_rZqiiDBP4XaLTT1ZaN6ykLm/view?usp=drive_link'
gdrive_link = 'https://drive.google.com/uc?id='
csv_dir = './csv'
zipped_file = './csv.zip'

download_csv(
    link_zipped_csv,
    gdrive_link,
    zipped_file
)

unzip_csv(
    zipped_file,
    csv_dir,
)

CSV file already downloaded!
CSV file already unzipped!


# CREATE LISTS

In [ ]:
# Read all the CSV files containing the Temperatures
features_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_02.csv'))
features_2 = pd.read_csv(os.path.join(prefix, 'csv/thermal_drift_features_lab_05_03.csv'))
features_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_04.csv'))
features_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_05.csv'))
features_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_06.csv'))

# Read all the CSV files containing the X1, Y1, Z1
targets_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_02.csv'))
targets_2 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_03.csv'))
targets_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_04.csv'))
targets_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_05.csv'))
targets_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_06.csv'))

features = [features_1, features_2, features_3, features_4, features_5]
targets = [targets_1,targets_2,targets_3,targets_4,targets_5]

print(features)
print(targets)

[       name  tags                 time  2"Tray1 Vacuum Sensor  \
0       daq   NaN  1714640400018369000               0.248703   
1       daq   NaN  1714640400095369000               0.248703   
2       daq   NaN  1714640400186397000               0.248627   
3       daq   NaN  1714640400395459000               0.248627   
4       daq   NaN  1714640400445367000               0.248627   
...     ...   ...                  ...                    ...   
716847  daq   NaN  1714690799641424000               0.248703   
716848  daq   NaN  1714690799742384000               0.248703   
716849  daq   NaN  1714690799839055000               0.248703   
716850  daq   NaN  1714690799890095000               0.248779   
716851  daq   NaN  1714690799995442000               0.248703   

        2"Tray2 Vacuum Sensor  2"Tray3 Vacuum Sensor  Avg Oven Temperature  \
0                    0.249084              -0.009865                     0   
1                    0.249084              -0.009713          

# DROP USELESS INFO

In [ ]:
for feature, target in zip(features, targets):
        feature.drop([
            "name", "tags",
            "2\"Tray1 Vacuum Sensor", "2\"Tray2 Vacuum Sensor", "2\"Tray3 Vacuum Sensor",
            "Avg Oven Temperature", "Chuck Temp [Cdeg]", "Chuck Temp2 [Cdeg]",
            "Chuck1 Vacuum Sensor", "Contrast", "Device State",
            "Dispenser1 Pressure Sensor", "Machine Room Temp", "Main Air", "Main Vacuum",
            "Oven Temperature", "PE_Rx", "PE_Ry", "PE_Rz", "PE_X1", "PE_Y1", "PE_Z1",
            "PUT1 Flow Sensor", "PUT2 Flow Sensor1", "PUT2 Flow Sensor2",
            "PUT2 Flow Sensor3", "PUT2 Flow Sensor4", "PUT2 Flow Sensor5",
            "Photodiode", "Pixel Power", "Preciser1 Vacuum Sensor",
            "Tec FIB1 Holder", "Tec FIB1 Plate", "Tec FIB2 Holder", "Tec FIB2 Plate",
            "Torque11","Torque2","Torque3","Torque4","Torque5","Torque6"
        ], axis=1, inplace=True)
        if 'name' in target.keys() and 'tags' in target.keys():
            target.drop(['name', 'tags'], axis=1, inplace=True)

print(features)
print(targets)

[                       time  Temp Sensor 1  Temp Sensor 2  Temp Sensor 3  \
0       1714640400018369000           2137           2164           2163   
1       1714640400095369000           2137           2164           2163   
2       1714640400186397000           2137           2164           2163   
3       1714640400395459000           2136           2164           2163   
4       1714640400445367000           2136           2164           2163   
...                     ...            ...            ...            ...   
716847  1714690799641424000           2095           2120           2134   
716848  1714690799742384000           2094           2120           2134   
716849  1714690799839055000           2094           2120           2134   
716850  1714690799890095000           2094           2120           2134   
716851  1714690799995442000           2094           2120           2134   

        Temp Sensor 4  
0                2159  
1                2159  
2             

# ELIMINATE NAN

In [ ]:
# Put X1, Y1, Z1 on the same row of X1 eliminating the NAN values
fixed_targets = [] # Create a list of target in which we put X1, Y1, Z1 in the same row
for target in targets:
    fixed_targets.append(transform_dataframe(target)) # iterate over target_1,2,3 ... and append in fixed_targets

print(fixed_targets)

[                     time            X1            Y1            Z1
0     1714640401106507000 -42506.390100  44830.277517  12931.601562
1     1714640412382008000 -42506.368407  44830.177972  12931.601562
2     1714640423632736000 -42506.327086  44830.179741  12931.523438
3     1714640434970393000 -42506.308733  44830.136638  12931.445312
4     1714640446143996000 -42506.280652  44830.203599  12931.289062
...                   ...           ...           ...           ...
4166  1714687151262569000 -42505.368696  44826.901348  12929.335938
4167  1714687162587275000 -42505.378471  44826.886557  12929.179688
4168  1714687173874593000 -42505.419544  44826.930953  12929.218750
4169  1714687185187085000 -42505.381234  44826.952204  12929.218750
4170  1714687196406018000 -42505.364203  44826.906375  12929.179688

[4171 rows x 4 columns],                      time            X1            Y1            Z1
0     1714687207558354000 -42505.406213  44826.890179  12929.179688
1     171468721884518

# MERGE ON CLOSEST TIME

In [ ]:
 # Merge of targets with features in one single dataframe
complete_numbers_list = [] # List of the table with columns that are numbers (0,1,2..) in which we unify both features and targets merging on closest time row
for fixed_target, feature in zip(fixed_targets, features):
    complete_numbers_list.append(merge_on_closest_time(fixed_target.reset_index(), feature.reset_index()))

print("The len of complete number list is equivalent to the number of file from 0...4:", len(complete_numbers_list))
print(complete_numbers_list)


# DIVIDING IN TRAIN E TEST

In [ ]:
trainig_number_list = []
testing_number_list = []
for i in range(len(complete_numbers_list)):
    part_numbers_list = complete_numbers_list[:i] + complete_numbers_list[i+1:]
    trainig_number_list.append(pd.concat(part_numbers_list))
    testing_number_list.append(complete_numbers_list[i])

print(trainig_number_list)
print(testing_number_list)

# Naming of columns

In [ ]:
training_list = []
testing_list = []
for training, testing in zip(trainig_number_list, testing_number_list):
    training_tmp = training.rename(columns={
        0: 'id',
        1: 'time',
        2: 'X1',
        3: 'Y1',
        4: 'Z1',
        5: 'to_remove',
        6: 'time_2',
        7: 'Temp1',
        8: 'Temp2',
        9: 'Temp3',
        10: 'Temp4'
        })
    training_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
    training_list.append(training_tmp)

    testing_tmp = testing.rename(columns={
        0: 'id',
        1: 'time',
        2: 'X1',
        3: 'Y1',
        4: 'Z1',
        5: 'to_remove',
        6: 'time_2',
        7: 'Temp1',
        8: 'Temp2',
        9: 'Temp3',
        10: 'Temp4'
        })
    testing_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
    testing_list.append(testing_tmp)

print(training_list)
print(testing_list)

In [ ]:
training = training_list[1]
testing = testing_list[1]

# Transform the training and test data in float
training.astype(float)
testing.astype(float)

# Take from dataframe the values of the columns of temperatures and positions saving into smallest dataframe of training/test
X_train = training[['Temp1','Temp2', 'Temp3', 'Temp4']]
Y_train = training[['X1', 'Y1', 'Z1']]
X_test = testing[['Temp1','Temp2', 'Temp3', 'Temp4']]
Y_test = testing[['X1', 'Y1', 'Z1']]

# Transform the X, Y from dataframe in numpy array both for test and train
X_train = X_train.values.astype(np.float32)
Y_train = Y_train.values.astype(np.float32)
X_test = X_test.values.astype(np.float32)
Y_test = Y_test.values.astype(np.float32)

# Do the gradient of the positions both for test and train
Y_train = my_gradient(Y_train, window_size=30)
Y_test = my_gradient(Y_test, window_size=30)

In [ ]:

datasetTest = thermal_dataset((X_test,Y_test), hyperparams['timesteps'], device)


In [ ]:
complete = True
autoencoder_only = False

# Load the weights
model.load(model_path, device=device)

# Set the model in evaluation mode
model.eval()

loss_model = 0 # value of the test loss for the model
loss_autoencoder = 0 #  value of the loss for the autoencoder
# Lists of predictions and true labels
y_pred_list = []
y_true_list = []
x_pred_list = []
x_true_list = []
# List of inference time
inference_time_list = []

with torch.no_grad():
    for i, data in enumerate(datasetTest):

        # Take input and label from the list [input, label]
        #print("Data: ", data)
        x, y = data       # tuple of tensor -> data = ([8,5,4], [8,3]) where 8 is batch size

        x = x.unsqueeze(0)
        y = y.unsqueeze(0)

        y_prec = y[:,0,:].unsqueeze(1)

        y_true = y[:,1,:].unsqueeze(1)

        # Counting the time for inference
        # Start counter
        start_time = time.time()

        # Make predictions
        # If we use ensemble model, then the model forward method should have y_true for the voting system
        # Else (LSTM only) the forward need only the input x
        if complete:
            y_true = y_true.detach()
            if autoencoder:
                (y_pred, y_pred_models), x_pred = model(x, y_prec)
                model.update_weights(y_true)
            else:
                y_pred, y_pred_models = model(x, y_prec)
            #print("y_pred:", y_pred.shape)
        else:
            y_pred = model(x)
            #print("y_pred:", y_pred.shape)

        # Stop counter
        end_time = time.time()
        inference_time = end_time - start_time # Compute the inference time
        inference_time_list.append(inference_time) # Append the inference time for each batch in the list

        # Compute the loss
        if not autoencoder_only:
            loss_model += loss_fn(y_pred, y_true).detach().item()

        if autoencoder:
            x = x.permute(0,2,1)
            loss_autoencoder += loss_fn(x_pred, x).detach().item()

        # Create the list of the y_true and y_pred
        # Transform the tensor(, device='cuda:0') in a list [] and summing the lists y_true_list = [ ...]
        y_true_list += y_true.squeeze(1).cpu().tolist()  # y_tru[:,1,:] because y_true[:,0,:] is the present value sended to ARIMA, while y_true[:,1,:] is the future value to be predicted
        y_pred_list += y_pred.squeeze(1).cpu().tolist()
        if autoencoder:
            x_true_list += x.cpu().tolist()
            x_pred_list += x_pred.cpu().tolist()

        #print("List of y_true:", y_true_list)
        #print("List of y_pred:", y_pred_list)

# Average Loss in testing
loss_model_avg = loss_model / len(datasetTest)   # tensor(value, device = 'cuda:0')
loss_autoencoder_avg = loss_autoencoder / len(datasetTest)   # tensor(value, device = 'cuda:0')

# Average of the training time
total_inference_time = sum(inference_time_list)
inference_time_avg = total_inference_time / len(inference_time_list)

In [ ]:

# Definition of the test metrics dictionary for the model
test_model_metrics = {
'rmse':[],
'rmse_ref':[],
'mae':[],
'mae_ref':[],
'r2':[],
'r2_ref':[],
'loss_avg': loss_model_avg,
'total_inference_time': total_inference_time,
'inference_time_avg': inference_time_avg
}

# Definition of the test metrics dictionary for the autoencoder
test_autoencoder_metrics = {
'rmse':[],
'rmse_ref':[],
'mae':[],
'mae_ref':[],
'r2':[],
'r2_ref':[],
'loss_avg': loss_autoencoder_avg,
}

print(y_true_list[0])

# Calculate the metrics
test_model_metrics = calculate_metrics(y_true_list, y_pred_list, test_model_metrics, train=False)
if autoencoder:
    test_autoencoder_metrics = calculate_metrics(x_true_list ,x_pred_list, test_autoencoder_metrics, autoencoder = True, train=False)

# We take all the path without "model.pt"
# (Ex. "./results/training_2025-01-28_11-11/model.pt" -> "./results/training_2025-01-28_11-11/")
results_path = model_path[:-8]

with open(results_path +'/test_model_metrics.json', 'w') as f:
    json.dump(test_model_metrics, f)
if autoencoder:
    with open(results_path +'/test_autoencoder_metrics.json', 'w') as f:
        json.dump(test_autoencoder_metrics, f)

print('Legend metrics:[X1,Y1,Z1]')
print("Test Model metrics:")
print(f"rmse: {test_model_metrics['rmse'][0]} , mae: {test_model_metrics['mae'][0]}, r2: {test_model_metrics['r2'][0]}")
print("Reference metrics(wrt. 0):")
print(f"rmse: {test_model_metrics['rmse_ref'][0]} , mae: {test_model_metrics['mae_ref'][0]}, r2: {test_model_metrics['r2_ref'][0]}")
if autoencoder:
    print("Test Autoencoder metrics:")
    print(f"rmse: {test_autoencoder_metrics['rmse'][0]} , mae: {test_autoencoder_metrics['mae'][0]}, r2: {test_autoencoder_metrics['r2'][0]}")
print("Others:")
print(f"Model Average Loss: {loss_model_avg}, Autoencoder Average Loss: {loss_autoencoder_avg}, Total Inference Time (All Dataset): {total_inference_time}, Average Inference Time: {inference_time_avg}")

if not 'NoOpModule' in str(type(model.ensemble)):
    print(f"Weights of the Ensemble models: {model.ensemble.weights.cpu().tolist()}")